In [1]:
import csv
import selenium
from config import *
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from pywinauto.keyboard import send_keys
from pywinauto.application import Application
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#---Despliegue del driver-----------------------
try:
    driver = webdriver.Chrome()
    driver.get(URL)
except Exception as e:
    print(f'Error al desplegar el driver: {e}')

In [3]:
#Función de ingreso al sistema Sade.
def login(username, password):
    try:
        XPATH_INPUT_USERNAME = '//input[@placeholder="Usuario/Cuil/Cuit"]'
        XPATH_INPUT_PASSWORD = '//input[@placeholder="Contraseña"]'
        XPATH_BUTTON_LOGIN   = '//button[@class="btn btn-default z-button"]'
        
        input_username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_INPUT_USERNAME)))
        input_password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_INPUT_PASSWORD)))
        button_loggin  = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_BUTTON_LOGIN)))
        
        input_username.send_keys(username)
        input_password.send_keys(password)
        
        button_loggin.click()
    except Exception as e:
        print(f'Ocurrió un error al intentar ingresar al sitio {e}')

In [4]:
#Función de nueva notificación electrónica.
def new_notification():
    try:
        XPATH_NOTIFICACTION_ELECTR = '//li[@title="Notificaciones Electronicas"]'
        XPATH_NEW_NOTIFICATION     = '//li[@title="Nueva Notificación"]'
        
        notification_electr        = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_NOTIFICACTION_ELECTR)))
        notification_electr.click()
        
        new_notification           = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, XPATH_NEW_NOTIFICATION)))
        new_notification.click()
    except Exception as e:
         print(f'Ocurrió un error al intentar generar una nueva notificación {e}')

In [5]:
#Función que ingresa todas las empresas.
def add_companies(companies):
    
    XPATH_INPUT_CUIT    = '//input[@name="cuit1"]'
    XPATH_BUTTONS       = '//td[@class="z-row-inner"]//button[@class="z-button"]'
    
    for razon, cuit in companies.items():
        try: 
            input_cuit    = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_INPUT_CUIT)))
            input_cuit.send_keys(cuit)
            
            buttons       = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_BUTTONS)))
            button_search = buttons[0]
            button_search.click()

            #Si encontró la empresa, avanzará:
            WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, f'//span[@class="z-label"][contains(text(), "{razon}")]')))
            
            button_add    = buttons[1]
            button_add.click()

            #Si se cargó la empresa, avanzará:
            WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH, f'//div[@class="z-listcell-content"][contains(text(), "{razon}")]')))
                
        except Exception as e:
            print(f"Ocurrió un error al intentar agregar la empresa con CUIT {cuit}: {e}")


In [6]:
#función que sube el pdf correspondiente.
def add_pdf(num_sap):
    input_upload_file =  driver.find_elements(By.XPATH, '//span[@class="z-radiogroup"]//span[@class="z-radio"]')[3]
    input_upload_file.click()
    sleep(1)
    
    button_add_file = driver.find_elements(By.XPATH, './/i[@class="glyphicon glyphicon-plus"]')[6]
    button_add_file.click()
    sleep(1)
    
    app = Application().connect(title_re='Abrir')
    path_pdf = fr'C:\Users\Usuario\OneDrive\Escritorio\DGFU\010 - Automatizacion Notificaciones TAD\resources\images\{num_sap}'
    app.Abrir.Edit.set_text(path_pdf)
    app.Abrir.Edit.type_keys('{ENTER}')

In [7]:
def send_notification(user):
    #Boton para seleccionar la opción "Quiero recibir un aviso cuando se firme la tarea"
    XPATH_BUTTON_IWANT   = '//div[@class="z-window-content"]//div[@class="z-hlayout"]//div[@class="z-hlayout-inner"]//input[@type="checkbox"]'
    XPATH_BUTTON_SEND    = '//div[@class="z-hlayout"]//div[@class="z-hlayout-inner"]//button[@class="z-button"]'
    XPATH_INPUT_USER     = './/input[@placeholder="Ingrese el usuario"]'
    XPATH_BUTTON_SEND_OK = './/tbody[@class="z-rows"]//tr[@class="GridLayoutNoBorder z-row"]//td[@class="z-row-inner"]//div[@class="z-row-content"]//button[@class="z-button"]'
    XPATH_BUTTON_NEXT    = './/button[@class="z-messagebox-button z-button"]'
    
    input_aviso = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_BUTTON_IWANT)))
    input_aviso = input_aviso[1]
    input_aviso.click()

    button_send = driver.find_elements(By.XPATH, XPATH_BUTTON_SEND)[9]
    button_send.click()

    input_send_username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_INPUT_USER)))
    input_send_username.send_keys(user)

    button_send_notif = driver.find_element(By.XPATH, XPATH_BUTTON_SEND_OK)
    button_send_notif.click()

    button_next =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_BUTTON_NEXT)))
    button_next.click()

In [8]:
#Funcion de envio de notificación por cada posición

def app(csv_path):

#---Ingreso a GEDO ------------------------------
    login(USERNAME, PASSWORD)
    
#---Nueva notificación---------------------------
    new_notification()


    
    with open(csv_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=';')
        rows   = list(reader)
        
        for row in tqdm(rows, desc="Enviando notificaciones"):
            num_sap  = row[1]
            position = row[2]

            add_companies(COMPANIES)
            #sleep(0.5)
        
            #Redactando el título y el cuerpo de la notificación----------------------------------------------------------------------------------------------------------------------------------------------
            XPATH_INPUT_TITLE  = '//div[@class="z-hlayout-inner"]//input[@class="z-textbox"]'
            XPATH_INPUT_BODY   = '//textarea[@placeholder="Ingresar texto de la notificación"]'
            XPATH_BUTTON_CHECK = '//tbody[@class="z-rows"]//tr[@class="z-row"]//td[@class="z-row-inner"]//div[@class="z-row-content"]//div[@class="z-hlayout"]//div[@class="z-hlayout-inner"]//button[@class="z-button"]'
            
            
            input_referencia = driver.find_element(By.XPATH, XPATH_INPUT_TITLE)
            input_referencia.send_keys(f"S/Faltante {position}")
            
            input_cuerpo     = driver.find_element(By.XPATH, XPATH_INPUT_BODY)
            input_cuerpo.send_keys(f"Por medio de la presente se lo intima a que en el plazo de 72 horas hábiles, desde recibida esta intimación, verifique e informe, conforme Articulo 5 de la Ley 4760, si la anomalía detectada sobre {position}, correspondiente a una cámara con tapa faltante sobre la acera), pertenece a la infraestructura de su empresa. En caso de corresponder, se lo intima en un plazo de 5 días hábiles a proceder a la reposición de tapa y/o anulación de la misma según corresponda, bajo el apercibimiento de aplicar el régimen sancionatorio previstas en la materia en el Régimen de Faltas de la Ciudad Autónoma de Buenos Aires (Ley N° 451). \n\n Asimismo, se lo intima a que en igual plazo presente formal descargo con muestras fotográficas en las que se verifique el cumplimiento de lo intimado en el primer párrafo de esta notificación ante la oficina de la Dirección General sita en Av. Martin García 346, 3°piso, en el horario de 10 a 15 hs o, mediante el correo electrónico de la Mesa de Entradas virtual mesaentrada.ssmaurb@buenosaires.gob.ar y gonzalogabrielramirez@buenosaires.gob.ar, todo ello, bajo apercibimiento de proceder al labrado de las correspondientes actas de comprobación, remoción y anulación de las instalaciones descriptas como irregulares. \n\n Se hace constar que el riesgo a terceros derivado de las instalaciones en deficiente estado habilita a esta dirección general a proceder cautelarmente respecto de las mismas, tomando la intervención física necesaria.")
            
            button_check     = driver.find_elements(By.XPATH, XPATH_BUTTON_CHECK)[0]
            button_check.click()

            #Adjuntando el pdf con las imágenes----------------------------------------------------------------------------------------------------------------------------------------------------------------
            add_pdf(num_sap)
            WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.XPATH, f'//div[@class="z-listcell-content"][contains(text(), "{num_sap}")]')))
                    
            #Función de envío----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            send_notification(USER)
                

In [9]:
app('resources/db/database.csv')

Enviando notificaciones: 100%|██████████| 3/3 [01:25<00:00, 28.43s/it]


In [10]:
#Rendimiento del código
#Test 
#Cantidad de notificaciones: 3
#Iteración 1: 4.8 minutos (1.6 minutos por notificación)
#Iteración 2: 3.12 minutos (1.04 minutos por notificación)
#Iteración 3: 2.46 minutos (49.2 segundos por Dnotificación)
#Iteración 4: 1.37 minutos (27.4 segundos por notificación)
#Iteración 5: 1.25 minutos (25 segundos por notificación)
